Basemodel: DistilBERT

In [32]:
!git clone https://github.com/drryodino246/csc542-gp-team164.git

fatal: destination path 'csc542-gp-team164' already exists and is not an empty directory.


In [33]:
!pip install datasets

In [34]:
from datasets import load_dataset
emotions = load_dataset('csv', data_files='/content/csc542-gp-team164/baseline.csv')

In [35]:
emotions_split = emotions['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = emotions_split['train']
test_dataset = emotions_split['test']

print(f"Training data size: {len(train_dataset)}")
print(f"Test data size: {len(test_dataset)}")

Training data size: 15347
Test data size: 3837


In [36]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt")

# Tokenize both the train and test datasets
train_encoded = train_dataset.map(tokenize, batched=True)
test_encoded = test_dataset.map(tokenize, batched=True)

# Rename 'updated_emotion' to 'label' for both train and test datasets
train_encoded = train_encoded.rename_column("updated_emotion", "label")
test_encoded = test_encoded.rename_column("updated_emotion", "label")

In [37]:
from transformers import AutoModelForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 4
id2label = {
    "0": "very negative",
    "1": "negative",
    "2": "positive",
    "3": "very positive"
}
label2id = {
    "very negative": 0,
    "negative": 1,
    "positive": 2,
    "very positive": 3
}

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id).to(device)

In [ ]:
pip install tensorboard

In [43]:
from transformers import TrainingArguments
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter

batch_size = 64
logging_steps = 500
model_name = f"{model_ckpt}-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f"{model_name}/logs",  # Directory for logs
    logging_steps=logging_steps,  # Log training progress every 'logging_steps'
    load_best_model_at_end=True,  # Load the best model based on evaluation metrics
    metric_for_best_model="f1",  # Track the F1 score for selecting the best model
    greater_is_better=True,  # We want higher F1 score, so set this to True
    disable_tqdm=False,  # Keep TQDM for progress bar
    report_to="tensorboard",  # Enable TensorBoard logging
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [44]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [45]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_encoded,
    eval_dataset=test_encoded,
    tokenizer=tokenizer
)

<ipython-input-45-d3a469f476ce>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [46]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:


Abort: 

In [ ]:
trainer.save_model()

In [ ]:
from tensorboard import notebook
notebook.display("logs")